# 🧪 Test Mistral Document Annotation / OCR

Este notebook permite probar rápidamente el Document Annotation de Mistral sobre documentos PDF/DOCX.

**Características:**
- Soporte para documentos de más de 8 páginas (chunking automático)
- Consolidación inteligente de chunks
- Múltiples modelos de datos predefinidos
- Visualización de resultados

## 1. Setup e Imports

In [1]:
import os
import sys
import json
import base64
import tempfile
import shutil
import logging
from pathlib import Path
from typing import Type, Optional, Any, Dict, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from contextlib import contextmanager

from pydantic import BaseModel, Field
from PyPDF2 import PdfReader, PdfWriter
from mistralai import Mistral
from mistralai.extra import response_format_from_pydantic_model
from dotenv import load_dotenv
from IPython.display import display, JSON, Markdown

# Agregar el directorio raíz al path para importar modelos
ROOT_DIR = Path(os.getcwd()).parent
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

# Cargar variables de entorno
load_dotenv(ROOT_DIR / ".env")

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print(f"✅ Root directory: {ROOT_DIR}")
print(f"✅ MISTRAL_API_KEY configurada: {'Sí' if os.getenv('MISTRAL_API_KEY') else 'No'}")

✅ Root directory: d:\Users\iromero\OneDrive - Grupo Procaps\Previos\Documents\pharma_ai\ma_change_control_agent
✅ MISTRAL_API_KEY configurada: Sí


## 2. Importar Modelos de Datos Disponibles

In [2]:
# Importar modelos existentes del proyecto
try:
    from src.models.change_control import ChangeControlModel
    from src.models.analytical_method_models import MetodoAnaliticoDA, MetodoAnaliticoCompleto
    from src.models.side_by_side_model import SideBySideModel
    from src.models.structured_test_model import TestSolution, TestSolutions
    print("✅ Modelos importados correctamente")
except ImportError as e:
    print(f"⚠️ Error importando modelos: {e}")
    print("Definiendo modelos básicos de fallback...")

# Diccionario de modelos disponibles para selección rápida
AVAILABLE_MODELS = {
    "change_control": ChangeControlModel,
    "analytical_method": MetodoAnaliticoDA,
    "analytical_method_complete": MetodoAnaliticoCompleto,
    "side_by_side": SideBySideModel,
    "test_solutions": TestSolutions,
}

print("\n📋 Modelos disponibles:")
for name, model in AVAILABLE_MODELS.items():
    print(f"   - {name}: {model.__name__}")

✅ Modelos importados correctamente

📋 Modelos disponibles:
   - change_control: ChangeControlModel
   - analytical_method: MetodoAnaliticoDA
   - analytical_method_complete: MetodoAnaliticoCompleto
   - side_by_side: SideBySideModel
   - test_solutions: TestSolutions


## 3. Funciones de Utilidad para Procesamiento de PDFs

In [3]:
@contextmanager
def prepare_pdf_document(document_path: str):
    """Asegura que el documento esté disponible como PDF, convirtiendo DOCX si es necesario."""
    if not document_path:
        raise ValueError("No se proporcionó la ruta del documento a procesar.")

    resolved_path = Path(document_path)
    if not resolved_path.exists():
        raise FileNotFoundError(f"El documento {document_path} no existe.")

    suffix = resolved_path.suffix.lower()
    if suffix == ".pdf":
        yield str(resolved_path)
        return

    if suffix == ".docx":
        temp_dir = tempfile.mkdtemp(prefix="doc_annotation_")
        pdf_output_path = os.path.join(temp_dir, resolved_path.with_suffix(".pdf").name)
        try:
            from docx2pdf import convert as docx_to_pdf_convert
            logger.info(f"Convirtiendo DOCX a PDF: {document_path}")
            docx_to_pdf_convert(str(resolved_path), pdf_output_path)

            if not os.path.exists(pdf_output_path):
                raise RuntimeError(f"No se generó el archivo PDF convertido para {document_path}.")

            yield pdf_output_path
        finally:
            shutil.rmtree(temp_dir, ignore_errors=True)
        return

    raise ValueError(f"Formato de archivo no soportado: {suffix}. Solo PDF o DOCX.")


def get_pdf_page_count(pdf_path: str) -> int:
    """Obtiene el número de páginas de un PDF."""
    try:
        with open(pdf_path, "rb") as pdf_file:
            reader = PdfReader(pdf_file)
            return len(reader.pages)
    except Exception as e:
        logger.error(f"Error contando páginas en {pdf_path}: {e}")
        return 0


def split_pdf_into_chunks(
    pdf_path: str, 
    max_pages_per_chunk: int = 8, 
    chunk_overlap_pages: int = 2
) -> list[str]:
    """Divide un PDF en chunks con overlap."""
    chunk_files: list[str] = []
    try:
        reader = PdfReader(pdf_path)
        total_pages = len(reader.pages)
        if total_pages == 0:
            return []

        overlap = max(chunk_overlap_pages, 0)
        chunk_size = max(max_pages_per_chunk, 1)
        step = max(chunk_size - overlap, 1)

        for start in range(0, total_pages, step):
            end = min(start + chunk_size, total_pages)
            chunk_writer = PdfWriter()
            for page_idx in range(start, end):
                chunk_writer.add_page(reader.pages[page_idx])

            with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as chunk_pdf:
                chunk_writer.write(chunk_pdf)
                chunk_files.append(chunk_pdf.name)

        logger.info(f"📄 PDF dividido en {len(chunk_files)} chunks (páginas: {total_pages}, chunk_size: {chunk_size}, overlap: {overlap})")
        return chunk_files
    except Exception as e:
        logger.error(f"Error dividiendo PDF: {e}")
        for chunk_file in chunk_files:
            try:
                os.unlink(chunk_file)
            except OSError:
                pass
        return []


def encode_pdf(pdf_path: str) -> str:
    """Codifica un PDF a base64."""
    try:
        with open(pdf_path, "rb") as pdf_file:
            return base64.b64encode(pdf_file.read()).decode('utf-8')
    except Exception as e:
        logger.error(f"Error codificando PDF {pdf_path}: {e}")
        return None

print("✅ Funciones de utilidad cargadas")

✅ Funciones de utilidad cargadas


## 4. Funciones de Procesamiento con Mistral OCR

In [4]:
def process_chunk(
    pdf_path: str, 
    extraction_model: Type[BaseModel] = None,
    chunk_retry_backoff_seconds: int = 5, 
    chunk_retry_attempts: int = 3,
    include_image_base64: bool = False
):
    """Procesa un chunk de PDF con Mistral OCR."""
    import time
    
    base64_pdf = encode_pdf(pdf_path)
    if not base64_pdf:
        return None
    
    api_key = os.getenv("MISTRAL_API_KEY")
    if not api_key:
        raise EnvironmentError("Defina MISTRAL_API_KEY en el entorno o en el archivo .env")
    
    ocr_client = Mistral(api_key=api_key, timeout_ms=300000)

    request_params = {
        "model": "mistral-ocr-latest",
        "document": {
            "type": "document_url",
            "document_url": f"data:application/pdf;base64,{base64_pdf}"
        },
        "include_image_base64": include_image_base64,
    }

    if extraction_model:
        try:
            request_params["document_annotation_format"] = response_format_from_pydantic_model(extraction_model)
            logger.info(f"🎯 Usando modelo de extracción: {extraction_model.__name__}")
        except Exception as exc:
            logger.warning(f"No se pudo generar schema pydantic: {exc}")

    last_exception: Optional[Exception] = None
    total_attempts = max(chunk_retry_attempts, 1)

    for attempt in range(1, total_attempts + 1):
        try:
            return ocr_client.ocr.process(**request_params)
        except Exception as exc:
            last_exception = exc
            if attempt >= total_attempts:
                break
            wait_seconds = chunk_retry_backoff_seconds * attempt
            logger.warning(f"Reintentando chunk después de error: {exc}. Intento {attempt}/{chunk_retry_attempts} en {wait_seconds}s")
            time.sleep(wait_seconds)

    logger.error(f"Error procesando chunk {pdf_path}: {last_exception}")
    return None


def process_document(
    pdf_path: str,
    extraction_model: Type[BaseModel] = None,
    max_pages_per_chunk: int = 8,
    chunk_overlap_pages: int = 2,
    max_workers: int = 4,
    include_image_base64: bool = False
) -> list:
    """Procesa un PDF con chunking automático si es necesario."""
    total_pages = get_pdf_page_count(pdf_path)
    logger.info(f"📄 Procesando PDF con {total_pages} páginas")
    
    if total_pages == 0:
        logger.error(f"No se pudieron leer páginas del PDF")
        return []
    
    # Si el documento es pequeño, procesar directamente
    if total_pages <= max_pages_per_chunk:
        logger.info(f"✅ Documento pequeño ({total_pages} páginas), procesando directamente")
        result = process_chunk(pdf_path, extraction_model, include_image_base64=include_image_base64)
        return [result] if result else []
    
    # Dividir en chunks y procesar en paralelo
    logger.info(f"📦 Documento grande ({total_pages} páginas), dividiendo en chunks...")
    chunk_files = split_pdf_into_chunks(
        pdf_path,
        max_pages_per_chunk=max_pages_per_chunk,
        chunk_overlap_pages=chunk_overlap_pages,
    )
    
    if not chunk_files:
        return []

    indexed_results: list[tuple[int, Any]] = []
    
    try:
        workers = max(1, min(max_workers, len(chunk_files)))
        logger.info(f"🚀 Procesando {len(chunk_files)} chunks con {workers} workers...")
        
        with ThreadPoolExecutor(max_workers=workers) as executor:
            future_map = {
                executor.submit(
                    process_chunk,
                    chunk_file,
                    extraction_model,
                    5,
                    3,
                    include_image_base64
                ): (idx, chunk_file)
                for idx, chunk_file in enumerate(chunk_files)
            }

            for future in as_completed(future_map):
                idx, chunk_file = future_map[future]
                try:
                    result = future.result()
                    if result:
                        indexed_results.append((idx, result))
                        logger.info(f"✅ Chunk {idx + 1}/{len(chunk_files)} procesado")
                except Exception as exc:
                    logger.error(f"❌ Error procesando chunk {idx + 1}: {exc}")
    finally:
        # Limpiar archivos temporales
        for chunk_file in chunk_files:
            try:
                os.unlink(chunk_file)
            except Exception:
                pass
    
    # Ordenar por índice
    indexed_results.sort(key=lambda item: item[0])
    return [result for _, result in indexed_results]

print("✅ Funciones de procesamiento Mistral cargadas")

✅ Funciones de procesamiento Mistral cargadas


## 5. Funciones de Consolidación de Chunks

In [5]:
def _merge_list_items(target_list: list, source_list: list):
    """Mergea listas cuidando duplicados y combinando elementos dict similares."""
    for item in source_list:
        if item in (None, [], {}, ""):
            continue

        if isinstance(item, dict):
            existing = next((t for t in target_list if isinstance(t, dict) and t == item), None)
            if existing is not None:
                _merge_chunk_data(existing, item)
                continue

        if item not in target_list:
            target_list.append(item)


def _merge_chunk_data(target: dict, source: dict):
    """Mergea datos de un chunk con el diccionario consolidado."""
    for key, value in source.items():
        if value in (None, [], {}, ""):
            continue

        if key not in target or target[key] in (None, [], {}):
            target[key] = value
            continue

        target_value = target[key]

        if isinstance(target_value, list) and isinstance(value, list):
            _merge_list_items(target_value, value)
        elif isinstance(target_value, dict) and isinstance(value, dict):
            _merge_chunk_data(target_value, value)
        elif isinstance(target_value, str) and isinstance(value, str):
            # Preferir el texto más largo para mantener contexto adicional
            if len(value.strip()) > len(target_value.strip()):
                target[key] = value
        else:
            target[key] = value


def consolidate_chunks_data(
    chunk_responses: list, 
    extraction_model: type[BaseModel] = None,
    return_raw: bool = False
) -> Dict[str, Any]:
    """Consolida los document_annotation de todos los chunks."""
    if not chunk_responses:
        logger.warning("No hay chunks para procesar")
        return None
    
    all_chunk_data = {}
    all_markdown = []
    
    for i, response in enumerate(chunk_responses):
        if not response:
            continue
        
        # Extraer markdown de cada página
        if hasattr(response, 'pages'):
            for page in response.pages:
                if hasattr(page, 'markdown') and page.markdown:
                    all_markdown.append(page.markdown)
        
        # Extraer document_annotation del chunk
        annotation_data = None
        if hasattr(response, 'document_annotation'):
            annotation_data = response.document_annotation
        elif isinstance(response, dict) and 'document_annotation' in response:
            annotation_data = response['document_annotation']
        
        if annotation_data:
            try:
                if isinstance(annotation_data, str):
                    chunk_data = json.loads(annotation_data)
                elif isinstance(annotation_data, dict):
                    chunk_data = annotation_data
                else:
                    chunk_data = json.loads(str(annotation_data))
                
                _merge_chunk_data(all_chunk_data, chunk_data)
                logger.debug(f"Merged chunk {i+1} data")
                
            except (json.JSONDecodeError, TypeError) as e:
                logger.warning(f"Error parsing chunk {i+1} annotation: {e}")
    
    result = {
        "consolidated_data": all_chunk_data,
        "markdown_combined": "\n\n---\n\n".join(all_markdown),
        "chunks_processed": len(chunk_responses),
    }
    
    # Intentar crear instancia del modelo Pydantic
    if all_chunk_data and extraction_model and not return_raw:
        try:
            model_instance = extraction_model(**all_chunk_data)
            result["model_instance"] = model_instance
            result["model_dict"] = model_instance.model_dump()
            logger.info(f"✅ Creada instancia de {extraction_model.__name__}")
        except Exception as e:
            logger.warning(f"⚠️ No se pudo crear instancia del modelo: {e}")
            result["model_instance"] = None
            result["model_dict"] = all_chunk_data
    else:
        result["model_instance"] = None
        result["model_dict"] = all_chunk_data
    
    return result

print("✅ Funciones de consolidación cargadas")

✅ Funciones de consolidación cargadas


## 6. Función Principal de Test

In [6]:
def test_document_annotation(
    document_path: str,
    model_name: str = None,
    custom_model: Type[BaseModel] = None,
    max_pages_per_chunk: int = 8,
    chunk_overlap_pages: int = 2,
    max_workers: int = 4,
    include_image_base64: bool = False,
    save_output: bool = True,
    output_dir: str = None
) -> Dict[str, Any]:
    """
    Función principal para testear Document Annotation de Mistral.
    
    Args:
        document_path: Ruta al documento PDF o DOCX
        model_name: Nombre del modelo predefinido ('change_control', 'analytical_method', etc.)
        custom_model: Modelo Pydantic personalizado (tiene prioridad sobre model_name)
        max_pages_per_chunk: Máximo de páginas por chunk
        chunk_overlap_pages: Páginas de overlap entre chunks
        max_workers: Workers para procesamiento paralelo
        include_image_base64: Incluir imágenes en base64 en la respuesta
        save_output: Guardar resultados en archivos JSON
        output_dir: Directorio para guardar outputs
    
    Returns:
        Dict con resultados consolidados
    """
    import time
    start_time = time.time()
    
    # Seleccionar modelo
    extraction_model = None
    if custom_model:
        extraction_model = custom_model
    elif model_name and model_name in AVAILABLE_MODELS:
        extraction_model = AVAILABLE_MODELS[model_name]
    
    print(f"\n{'='*60}")
    print(f"🔬 TEST MISTRAL DOCUMENT ANNOTATION")
    print(f"{'='*60}")
    print(f"📄 Documento: {document_path}")
    print(f"🎯 Modelo: {extraction_model.__name__ if extraction_model else 'Sin modelo (solo OCR)'}")
    print(f"📦 Config: {max_pages_per_chunk} páginas/chunk, {chunk_overlap_pages} overlap")
    print(f"{'='*60}\n")
    
    results = {
        "document_path": document_path,
        "model_used": extraction_model.__name__ if extraction_model else None,
        "config": {
            "max_pages_per_chunk": max_pages_per_chunk,
            "chunk_overlap_pages": chunk_overlap_pages,
            "max_workers": max_workers
        },
        "success": False,
        "error": None
    }
    
    try:
        with prepare_pdf_document(document_path) as pdf_path:
            results["total_pages"] = get_pdf_page_count(pdf_path)
            
            # Procesar documento
            chunk_responses = process_document(
                pdf_path=pdf_path,
                extraction_model=extraction_model,
                max_pages_per_chunk=max_pages_per_chunk,
                chunk_overlap_pages=chunk_overlap_pages,
                max_workers=max_workers,
                include_image_base64=include_image_base64
            )
            
            results["chunks_processed"] = len(chunk_responses)
            
            # Consolidar resultados
            consolidated = consolidate_chunks_data(chunk_responses, extraction_model)
            
            results["consolidated_data"] = consolidated["model_dict"]
            results["markdown_combined"] = consolidated["markdown_combined"]
            results["model_instance"] = consolidated["model_instance"]
            results["success"] = True
            
    except Exception as e:
        results["error"] = str(e)
        logger.error(f"❌ Error: {e}")
    
    elapsed_time = time.time() - start_time
    results["elapsed_time_seconds"] = round(elapsed_time, 2)
    
    # Guardar resultados
    if save_output and results["success"]:
        output_dir = output_dir or str(ROOT_DIR / "notebooks" / "outputs")
        os.makedirs(output_dir, exist_ok=True)
        
        doc_name = Path(document_path).stem
        
        # Guardar JSON estructurado
        json_path = os.path.join(output_dir, f"{doc_name}_structured.json")
        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(results["consolidated_data"], f, indent=2, ensure_ascii=False, default=str)
        
        # Guardar Markdown
        md_path = os.path.join(output_dir, f"{doc_name}_ocr.md")
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(results["markdown_combined"])
        
        print(f"\n💾 Resultados guardados en:")
        print(f"   - JSON: {json_path}")
        print(f"   - Markdown: {md_path}")
    
    # Resumen
    print(f"\n{'='*60}")
    print(f"📊 RESUMEN")
    print(f"{'='*60}")
    print(f"✅ Éxito: {results['success']}")
    print(f"📄 Páginas totales: {results.get('total_pages', 'N/A')}")
    print(f"📦 Chunks procesados: {results.get('chunks_processed', 'N/A')}")
    print(f"⏱️ Tiempo: {results['elapsed_time_seconds']}s")
    if results.get('error'):
        print(f"❌ Error: {results['error']}")
    print(f"{'='*60}\n")
    
    return results

print("✅ Función principal de test cargada")

✅ Función principal de test cargada


## 7. Funciones de Visualización

In [7]:
def show_results(results: Dict[str, Any], show_markdown: bool = False, max_json_depth: int = 3):
    """Muestra los resultados de forma visual."""
    if not results.get("success"):
        print(f"❌ El procesamiento falló: {results.get('error')}")
        return
    
    print("\n📋 DATOS ESTRUCTURADOS EXTRAÍDOS:")
    print("-" * 40)
    
    # Mostrar JSON interactivo
    display(JSON(results["consolidated_data"], expanded=False))
    
    if show_markdown and results.get("markdown_combined"):
        print("\n📝 MARKDOWN OCR (primeros 2000 caracteres):")
        print("-" * 40)
        display(Markdown(results["markdown_combined"][:2000] + "..."))


def show_model_fields(model_name: str = None, custom_model: Type[BaseModel] = None):
    """Muestra los campos de un modelo de datos."""
    model = custom_model or AVAILABLE_MODELS.get(model_name)
    if not model:
        print(f"❌ Modelo no encontrado: {model_name}")
        return
    
    print(f"\n📋 Campos del modelo {model.__name__}:")
    print("-" * 40)
    
    for field_name, field_info in model.model_fields.items():
        field_type = str(field_info.annotation).replace("typing.", "")
        required = "✅" if field_info.is_required() else "⬜"
        desc = field_info.description[:80] + "..." if field_info.description and len(field_info.description) > 80 else (field_info.description or "")
        print(f"{required} {field_name}: {field_type}")
        if desc:
            print(f"   └─ {desc}")

print("✅ Funciones de visualización cargadas")

✅ Funciones de visualización cargadas


---

# 🧪 ZONA DE PRUEBAS

Modifica las celdas siguientes para probar tus documentos.

## Ejemplo 1: Ver campos de un modelo

In [8]:
# Ver campos disponibles de un modelo
show_model_fields("change_control")


📋 Campos del modelo ChangeControlModel:
----------------------------------------
⬜ codigo_solicitud: Optional[str]
   └─ Código de la solicitud de cambio, normalmente se encuentra en el encabezado del ...
⬜ fecha_solicitud: Optional[str]
   └─ Fecha de la solicitud.
⬜ nombre: Optional[str]
   └─ Nombre de la persona que presenta el cambio.
⬜ cargo: Optional[str]
   └─ Cargo de la persona que presenta el cambio. Puede ser Analistas, Jefes, Coordina...
⬜ titulo: Optional[str]
   └─ Título del cambio. Puede ser el nombre de un producto o declarar el nombre del m...
⬜ fecha_aprobacion: Optional[str]
   └─ Fecha de aprobación del cambio.
✅ descripcion_cambio: List[src.models.change_control.DescripcionCambio]
   └─ Listado de descripciones de los diferentes cambios en las pruebas del método ana...
⬜ cliente: Optional[str]
   └─ Nombre del cliente. Se encuentra cerca del string 'CLIENTE'.
⬜ centro: Optional[str]
   └─ Nombre del centro. Se encuentra cerca del string 'CENTRO'.
⬜ codigos_produ

In [9]:
show_model_fields("analytical_method")


📋 Campos del modelo MetodoAnaliticoDA:
----------------------------------------
⬜ apis: Optional[List[str]]
   └─ Listado de ingredientes activos (APIs) del producto, si se indican en el método.
⬜ tipo_metodo: Optional[src.models.analytical_method_models.TipoMetodo]
   └─ Tipo de método según el encabezado o portada. Seleccionar una de las opciones de...
⬜ nombre_producto: Optional[str]
   └─ Nombre comercial/técnico del producto en el encabezado. Mantener potencia si est...
⬜ numero_metodo: Optional[str]
   └─ Identificador del método indicado como 'Método No', 'CÓDIGO', 'MÉTODO', etc. Ej....
⬜ version_metodo: Optional[str]
   └─ Versión del método indicada en el encabezado o portada. Ej.: '01'. Solo el valor...
⬜ codigo_producto: Optional[str]
   └─ Código del producto en el encabezado o tabla de alcance (columna 'Código'/'Códig...
⬜ tabla_de_contenidos: Optional[List[str]]
   └─ Extracción EXHAUSTIVA y LITERAL de la estructura jerárquica del documento. Instr...
⬜ objetivo: Optional

## Ejemplo 2: Probar un documento con modelo predefinido

In [14]:
# ⚠️ MODIFICA ESTA RUTA con tu documento
DOCUMENT_PATH = r"D:/Users/iromero/OneDrive - Grupo Procaps/Portafolio NTF/16 - I&D 4.0/21. Template migration workflow/Caso Gestavit DHA/ANEXO NAPROXENO SODICO 100000346 -rotated-real.pdf"

# Modelos disponibles: 'change_control', 'analytical_method', 'analytical_method_complete', 'side_by_side', 'test_solutions'
MODEL_NAME = "side_by_side"

# Ejecutar test
results = test_document_annotation(
    document_path=DOCUMENT_PATH,
    model_name=MODEL_NAME,
    max_pages_per_chunk=8,
    chunk_overlap_pages=2,
    save_output=True
)

2025-12-04 16:23:14,429 - INFO - 📄 Procesando PDF con 30 páginas
2025-12-04 16:23:14,430 - INFO - 📦 Documento grande (30 páginas), dividiendo en chunks...


2025-12-04 16:23:14,540 - INFO - 📄 PDF dividido en 5 chunks (páginas: 30, chunk_size: 8, overlap: 2)
2025-12-04 16:23:14,542 - INFO - 🚀 Procesando 5 chunks con 4 workers...



🔬 TEST MISTRAL DOCUMENT ANNOTATION
📄 Documento: D:/Users/iromero/OneDrive - Grupo Procaps/Portafolio NTF/16 - I&D 4.0/21. Template migration workflow/Caso Gestavit DHA/ANEXO NAPROXENO SODICO 100000346 -rotated-real.pdf
🎯 Modelo: SideBySideModel
📦 Config: 8 páginas/chunk, 2 overlap



2025-12-04 16:23:14,617 - INFO - 🎯 Usando modelo de extracción: SideBySideModel
2025-12-04 16:23:14,677 - INFO - 🎯 Usando modelo de extracción: SideBySideModel
2025-12-04 16:23:14,723 - INFO - 🎯 Usando modelo de extracción: SideBySideModel
2025-12-04 16:23:14,749 - INFO - 🎯 Usando modelo de extracción: SideBySideModel
2025-12-04 16:23:51,742 - INFO - HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-12-04 16:23:51,748 - INFO - ✅ Chunk 2/5 procesado
2025-12-04 16:23:51,781 - INFO - 🎯 Usando modelo de extracción: SideBySideModel
2025-12-04 16:23:57,399 - INFO - HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-12-04 16:23:57,407 - INFO - ✅ Chunk 4/5 procesado
2025-12-04 16:24:14,788 - INFO - HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-12-04 16:24:14,916 - INFO - ✅ Chunk 3/5 procesado
2025-12-04 16:24:34,663 - INFO - HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"
2025-12-04 16:24:34,669 - INFO - ✅ 


💾 Resultados guardados en:
   - JSON: d:\Users\iromero\OneDrive - Grupo Procaps\Previos\Documents\pharma_ai\ma_change_control_agent\notebooks\outputs\ANEXO NAPROXENO SODICO 100000346 -rotated-real_structured.json
   - Markdown: d:\Users\iromero\OneDrive - Grupo Procaps\Previos\Documents\pharma_ai\ma_change_control_agent\notebooks\outputs\ANEXO NAPROXENO SODICO 100000346 -rotated-real_ocr.md

📊 RESUMEN
✅ Éxito: True
📄 Páginas totales: 30
📦 Chunks procesados: 5
⏱️ Tiempo: 86.24s



In [15]:
# Visualizar resultados
show_results(results, show_markdown=True)


📋 DATOS ESTRUCTURADOS EXTRAÍDOS:
----------------------------------------


<IPython.core.display.JSON object>


📝 MARKDOWN OCR (primeros 2000 caracteres):
----------------------------------------


![img-0.jpeg](img-0.jpeg)

Página 1 de 31

# ANEXO I

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SODICO 100000346

|  MÉTODO DE ANÁLISIS | MODIFICACIÓN PROPUESTA  |
| --- | --- |
|  **MÉTODO DE ANÁLISIS 100000346**
10-0514 Versión 01
(VERSIÓN ACTUAL) | **MÉTODO DE ANÁLISIS 100000346**
10-0514 Versión 02
(NUEVA VERSIÓN)  |
|  **DESARROLLO** |   |
|  **NOMBRE DEL MATERIAL:** NAPROXEN SODICO. | **NOMBRE DEL MATERIAL:** NAPROXENO SODICO.  |
|  **DESCRIPCIÓN (INTERNA)** | **DESCRIPCIÓN (USP)**  |
|  Tomar una porción de la muestra, extenderla sobre una caja de Petri y observar sus características. Debe estar libre de material extraño. | Tomar una porción de la muestra, extenderla sobre una caja de Petri y observar sus características. Debe estar libre de material extraño.  |
|  Criterio de aceptación: Polvo de color blanco o crema, untuoso al tacto, libre de material extraño. | Criterio de aceptación: Polvo cristalino de color blanco a cremoso.  |
|  **PUNTO DE FUSIÓN (USP)** | **PUNTO DE FUSIÓN (USP)**  |
|  Colocar un poco de la muestra en un tubo capilar, llevar a un fusiómetro debidamente calibrado y determinar el punto de fusión. Funde cerca de 255°C con descomposición. | Colocar un poco de la muestra en un tubo capilar, llevar a un fusiómetro debidamente calibrado y determinar el punto de fusión. Funde cerca de 255°C con descomposición.  |
|  Criterio de aceptación: Funde aproximadamente a 255°C. | Criterio de aceptación: Funde aproximadamente a 255°C, con descomposición.  |
|  **IDENTIFICACIÓN A (IR) (Cofa)** | **IDENTIFICACIÓN A (IR) (USP)**  |
|  Colocar una pequeña cantidad de la muestra, previamente pulverizada y mezclada con bromuro de potasio en un mortero, en el dispositivo de Reflectancia Total Atenuada (ATR). Obtener el espectro IR de la muestra y compararlo con el espectro obtenido a partir de un estándar de referencia bajo las mismas condiciones de tratamiento y equipo. Verificar que el espect...

## Ejemplo 3: Probar solo OCR (sin modelo estructurado)

In [16]:
# ⚠️ MODIFICA ESTA RUTA con tu documento
DOCUMENT_PATH = r"D:/Users/iromero/OneDrive - Grupo Procaps/Portafolio NTF/16 - I&D 4.0/21. Template migration workflow/Caso Gestavit DHA/ANEXO NAPROXENO SODICO 100000346 -rotated-real.pdf"

# Sin modelo = solo OCR a Markdown
results_ocr = test_document_annotation(
    document_path=DOCUMENT_PATH,
    model_name=None,  # Sin modelo
    max_pages_per_chunk=50,
    save_output=True
)

2025-12-04 16:24:40,717 - INFO - 📄 Procesando PDF con 30 páginas
2025-12-04 16:24:40,719 - INFO - ✅ Documento pequeño (30 páginas), procesando directamente



🔬 TEST MISTRAL DOCUMENT ANNOTATION
📄 Documento: D:/Users/iromero/OneDrive - Grupo Procaps/Portafolio NTF/16 - I&D 4.0/21. Template migration workflow/Caso Gestavit DHA/ANEXO NAPROXENO SODICO 100000346 -rotated-real.pdf
🎯 Modelo: Sin modelo (solo OCR)
📦 Config: 50 páginas/chunk, 2 overlap



2025-12-04 16:24:52,455 - INFO - HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 200 OK"



💾 Resultados guardados en:
   - JSON: d:\Users\iromero\OneDrive - Grupo Procaps\Previos\Documents\pharma_ai\ma_change_control_agent\notebooks\outputs\ANEXO NAPROXENO SODICO 100000346 -rotated-real_structured.json
   - Markdown: d:\Users\iromero\OneDrive - Grupo Procaps\Previos\Documents\pharma_ai\ma_change_control_agent\notebooks\outputs\ANEXO NAPROXENO SODICO 100000346 -rotated-real_ocr.md

📊 RESUMEN
✅ Éxito: True
📄 Páginas totales: 30
📦 Chunks procesados: 1
⏱️ Tiempo: 11.77s



In [17]:
# Ver el markdown extraído
if results_ocr.get("success"):
    display(Markdown(results_ocr["markdown_combined"])) #display(Markdown(results_ocr["markdown_combined"][:5000]))

![img-0.jpeg](img-0.jpeg)

Página 1 de 31

# ANEXO I

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SODICO 100000346

|  MÉTODO DE ANÁLISIS | MODIFICACIÓN PROPUESTA  |
| --- | --- |
|  **MÉTODO DE ANÁLISIS 100000346**
10-0514 Versión 01
(VERSIÓN ACTUAL) | **MÉTODO DE ANÁLISIS 100000346**
10-0514 Versión 02
(NUEVA VERSIÓN)  |
|  **DESARROLLO** |   |
|  **NOMBRE DEL MATERIAL:** NAPROXEN SODICO. | **NOMBRE DEL MATERIAL:** NAPROXENO SODICO.  |
|  **DESCRIPCIÓN (INTERNA)** | **DESCRIPCIÓN (USP)**  |
|  Tomar una porción de la muestra, extenderla sobre una caja de Petri y observar sus características. Debe estar libre de material extraño. | Tomar una porción de la muestra, extenderla sobre una caja de Petri y observar sus características. Debe estar libre de material extraño.  |
|  Criterio de aceptación: Polvo de color blanco o crema, untuoso al tacto, libre de material extraño. | Criterio de aceptación: Polvo cristalino de color blanco a cremoso.  |
|  **PUNTO DE FUSIÓN (USP)** | **PUNTO DE FUSIÓN (USP)**  |
|  Colocar un poco de la muestra en un tubo capilar, llevar a un fusiómetro debidamente calibrado y determinar el punto de fusión. Funde cerca de 255°C con descomposición. | Colocar un poco de la muestra en un tubo capilar, llevar a un fusiómetro debidamente calibrado y determinar el punto de fusión. Funde cerca de 255°C con descomposición.  |
|  Criterio de aceptación: Funde aproximadamente a 255°C. | Criterio de aceptación: Funde aproximadamente a 255°C, con descomposición.  |
|  **IDENTIFICACIÓN A (IR) (Cofa)** | **IDENTIFICACIÓN A (IR) (USP)**  |
|  Colocar una pequeña cantidad de la muestra, previamente pulverizada y mezclada con bromuro de potasio en un mortero, en el dispositivo de Reflectancia Total Atenuada (ATR). Obtener el espectro IR de la muestra y compararlo con el espectro obtenido a partir de un estándar de referencia bajo las mismas condiciones de tratamiento y equipo. Verificar que el espectro de absorción infrarrojo de la muestra sea similar al del estándar. | Colocar una pequeña cantidad de la muestra, previamente pulverizada en un mortero, en el dispositivo de Reflectancia Total Atenuada (ATR). Obtener el espectro IR de la muestra y compararlo con el espectro obtenido a partir de un estándar de referencia bajo las mismas condiciones de tratamiento y equipo. Verificar que el espectro de absorción infrarrojo de la muestra sea similar al del estándar.  |

---

![img-1.jpeg](img-1.jpeg)

Página 2 de 31

Criterio de aceptación: El espectro de absorción infrarrojo de la preparación obtenida a apartir de la muestra de prueba, presenta máximos solo a las mismas longitudes de onda que el de una preparación similar del estándar de referencia correspondiente.

# IDENTIFICACIÓN B (UV) (USP)

## Solución Estándar

Transferir 25.0 mg del Estándar de Naproxen Sódico a una volumétrica de 100 ml, disolver con 20 ml de Metanol, aforar a volumen con Metanol y mezclar. Transferir una alícuota de 5 ml a una volumétrica de 50 ml, aforar a volumen con Metanol y mezclar.

## Solución Muestra

Transferir 25.0 mg de la muestra a una volumétrica de 100 ml, disolver con 20 ml de Metanol, aforar a volumen con Metanol y mezclar. Transferir una alícuota de 5 ml a una volumétrica de 50 ml, aforar a volumen con Metanol y mezclar.

## Procedimiento

Determinar las absorbencias de las soluciones finales de estándar y muestra en un espectrofotómetro debidamente calibrado usando una celda de 1 cm de paso óptico, a una longitud de onda de 272 nm y utilizando metanol, como blanco.

Determinar las absortividades mediante la siguiente fórmula:

$$A = \frac{Abs \times 1000 \times 100 \times 50}{W \times 5 \times 100 - 5 \mu m}$$

Donde

Abs: Absorbancia de muestra o estándar

W: Peso de muestra o estándar

5μm: Porcentaje de humedad.

Criterio de aceptación: El espectro IR de la preparación de la muestra presenta valores máximos solo a los mismos números de onda que los del Estándar de Referencia correspondiente, preparado apropiadamente.

# IDENTIFICACIÓN B (HPLC) (USP)

Comparar los cromatogramas de la solución estándar y la solución muestra obtenidos en el ensayo de Impureza Enantiomérica.

Criterio de aceptación: El tiempo de retención del pico principal de la Solución muestra corresponde al del pico de Naproxeno en la Solución de aptitud del sistema, según se obtiene en la Prueba de Pureza Enantiomérica.

---

![img-2.jpeg](img-2.jpeg)

**ANEXO I**

**CONTROL DE CAMBIO SC-25-777**

**CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA**

**NAPROXENO SODICO**

**100000346**

Página 3 de 31

Con la absortividad del estándar de entre el rango en el que debe estar la absortividad de la muestra mediante la siguiente fórmula:

$$
\text{Rango} = \frac{A \cdot \text{std}}{100} \pm A \cdot \frac{\text{std} \cdot 3}{100}
$$

**Criterio de aceptación:** Las absortividades a 272 nm calculadas con respecto a la n sustancia seca, no difieren en más de 3%.

### **ROTACIÓN ESPECÍFICA (USP) (CoFa)**

#### **Solución muestra**

Disolver 1.25 g de muestra en hidróxido de sodio 0.1N hasta completar 25 ml.

#### **Procedimiento**

La medición de la Rotación Específica se realiza a 589 nm y 25°C. Cuando se emplea un polarímetro fotoeléctrico, se hace una sola medición corregida por el blanco de disolvente. Cuando se emplea un polarímetro visual, se utiliza el promedio de no menos de 5 determinaciones, corregidas por la lectura del mismo tubo con un blanco de disolvente. La temperatura, que se aplica a la solución muestra, debe mantenerse con una aproximación de 0.5°C del valor establecido. Emplear la misma celda para la muestra y el blanco. Mantener la misma orientación angular de la celda en cada lectura. Colocar la celda de tal manera que la luz la atraviese en la misma dirección cada vez. La Rotación Específica, se calcula con respecto a la sustancia seca.

**Criterio de aceptación:** 15.3 / 17.0.

### **PERDIDA POR SECADO (USP) (CoFa)**

Transferir cuidadosamente entre 1.0 y 2.0 g de la muestra a una cápsula de porcelana previamente tarada y someter a secamiento en un desecador al vacío a 105°C por 3 horas, retirar de la estufa, llevar a un desecador por espacio de una hora y pesar.

### **PERDIDA POR SECADO (USP)**

Tarar un frasco para pesada adecuado con tapón de vidrio o una caja petri que se haya secado durante aproximadamente 30 minutos bajo las mismas condiciones que deben emplearse en la determinación y enfriado a temperatura ambiente en un desecador.

---

![img-3.jpeg](img-3.jpeg)

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO 100000346

#### Calcular el porcentaje de pérdida obtenido por la fórmula:

$$ \% \frac{W_{mf}}{W_{mi}} \times 100 $$

**Donde:**

- **Wmi:** Es el peso inicial de la muestra. (g)
- **Wmf:** Es la diferencia entre el peso inicial de muestra y el peso de la muestra después del tratamiento (g)
- **Criterio de aceptación:** No pierde más de 1.0% de su peso

Triturar la muestra antes de pesarla si esta estuviera en forma de partículas grandes, reducir el tamaño de las partículas aproximadamente a 2 mm, transferir cuidadosamente entre 1.0 y 2.0 g de la muestra. Colocar la muestra a analizar en el frasco o en la caja petri, volver a colocar el tapón o la tapa y pesar con exactitud el frasco con tapón y/o la caja petri con la tapa y el contenido. Distribuir la muestra a analizar uniformemente como sea posible, agitando suavemente hacia los lados, hasta lograr una profundidad por lo general de aproximadamente 5 mm y no más de 10 mm. Colocar el frasco y/o caja petri cargado en un desecador al vacío, retirar el tapón o tapa y dejarlo también en el desecador, dejar secar a 105°C ± 2 por 3 horas, retirar del desecador y pesar.

**Cálculos**

- Calcular el porcentaje de pérdida obtenido por la fórmula:

$$ \% \frac{W_{mf}}{W_{mi}} \times 100 $$

**Donde:**

- **Wmi:** Es el peso inicial de la muestra. (g)
- **Wmf:** Es la diferencia entre el peso inicial de muestra y el peso de la muestra después del tratamiento. (g)
- **Criterio de aceptación:** No más de 1.0%

## NAPROXENO LIBRE (USP)

Transferir 5.0 g de muestra a un embudo de separación y disolver con 25 mL de agua, extraer la solución con tres porciones de 15 mL de Cloroformo. Evaporar los extractos combinados en un baño de maría hasta sequedad. Disolver el residuo en 10 mL de una mezcla de Metanol - Agua (3:1) previamente neutralizado con hidroxido de sodio 0.1 N, hasta el punto final de la Fenolftaleína. Adicionar Fenolftaleína SR y titular con hidroxido de sodio 0.1 N.

## LIMITE DE NAPROXENO LIBRE (USP)

**Soluciones**

### Hidróxido de Sodio 1 N SV

Disolver 162 g de hidroxido de sodio en 150 mL de agua exenta de dióxido de carbono, enfriar la solución a temperatura ambiente y filtrar a través de un papel de filtro endurecido. Preparar la solución 1 N transfiriendo 54.5 mL del filtrado transparente a un envase de poliolefina impermeable y diluir con agua exenta de dióxido de carbono hasta 1,000 mL.

---

![img-4.jpeg](img-4.jpeg)

Página 5 de 31

Criterio de aceptación: No consume más de 2.2 mL de hidróxido de sodio 0.1N (1.0%)

### Hidróxido de Sodio 0.1 N SV

Transferir 100 mL de hidróxido de sodio 1 N SV a un matraz volumétrico de 1,000 mL. Diluir con agua exenta de dióxido de carbono a volumen.

### Fenolftaleína SR

Disolver 1 g de fenolftaleína en 100 mL de alcohol.

### Mezcla de Metanol y Agua (3:1)

Transferir 300 mL de metanol a un recipiente adecuado, luego agregar lentamente 100 mL de agua, mezclar suavemente.

### Procedimiento

Transferir 5.0 g de muestra a un embudo de separación y disolver con 25.0 mL de agua, extraer la solución con tres porciones de 15.0 mL de Cloroformo. Evaporar los extractos combinados en un baño de vapor hasta sequedad. Disolver el residuo en 10.0 mL de la Mezcla de Metanol y Agua (3:1) previamente neutralizado con hidróxido de sodio 0.1 N, hasta el punto final de la Fenolftaleína. Adicionar Fenolftaleína SR y titular con hidróxido de sodio 0.1 N.

Criterio de aceptación: No se consume más de 2.2 mL (1.0%)

### METALES PESADOS (Método I) (USP)

#### Solución amortiguadora de acetato de pH 3.5

Disolver 25.0 g de acetato de amonio en 25 mL de agua y adicionar 38 mL de ácido clorhídrico 6 N. Ajustar el pH, si es necesario, con hidróxido de amonio 6 N o ácido clorhídrico 6 N hasta pH 3.5; diluir con agua hasta 100 mL y mezclar.

#### Tioacetamida SR

Disolver 4.0 g de tioacetamida en 100 mL agua.

#### Glicerina base SR

### Se elimina la prueba

Se incluye la siguiente nota:

**NOTA:** De acuerdo a la declaración recibida de parte del fabricante, PROCAPS pudo establecer que el material no contiene impurezas elementales o si las contiene el aporte de estas cumplen con los criterios establecidos para la exposición por vía oral. Por lo anterior Cumple requerimientos USP <232>

---

![img-5.jpeg](img-5.jpeg)

Página 6 de 31

A 200.0 g de glicerina, adicionar agua hasta obtener un peso total de 235.0 g. Agregar 140 mL de hidróxido de sodio 1 N, 50 mL de agua y mezclar.

### Tioacetamida - Glicerina base SR

Mezclar 0.2 mL de Tioacetamida SR y 1 mL de Glicerina base SR y calentar en un baño de agua hirviente por 20 segundos. Usar esta mezcla inmediatamente.

### Solución madre de nitrato de plomo

Disolver 159.8 mg de nitrato de plomo en 100 mL de agua a la que se le ha agregado 1 mL de ácido nítrico, luego diluir con agua hasta 1000 mL. Preparar y almacenar esta solución en recipientes de vidrio libres de sales de plomo solubles.

### Solución estándar de plomo

En el día de uso, diluir con agua 10 mL de la Solución madre de nitrato de plomo hasta 100 mL. Cada mL de esta solución contiene el equivalente a 10 µg de plomo. Una solución de comparación preparada sobre la base de 100 µL de solución estándar de plomo por g de sustancia en análisis contiene el equivalente a 1 parte de plomo por millón de partes de la sustancia en análisis.

### Preparación estándar

En un tubo de comparación de 50 mL adicional 2 mL exactos de Solución estándar de plomo (20 µg de plomo) y diluir con agua hasta 25 mL. Ajustar a pH entre 3.0 y 4.0 con ácido acético 1 N ó hidróxido de amonio 6 N, según sea el caso, usando un pH-metro o un papel indicador de pH de rango estrecho (preferiblemente con límites de 2 a 5) como indicador externo. Diluir con agua hasta 40 mL y mezclar.

### Preparación de prueba

Transferir 1.0 g de muestra a un embudo de separación y disolver con 20 mL de agua, adicionar 5 mL de Acido Clorhídrico 1 N y extraer sucesivamente con 2 porciones de 20 mL y una porción de 10 mL de Cloruro de Metileno. Descartar la fase de Cloruro de Metileno y usar la fase acuosa para el ensayo.

---

![img-6.jpeg](img-6.jpeg)

Página 7 de 31

# ANEXO I

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO

##### 100000346

## Preparación control

En un tercer tubo de comparación de color de 50 mL, adicionar 25 mL de una solución preparada según se indica en la Preparación de prueba y agregar 2 mL de la Solución estándar de plomo. Ajustar a pH entre 3.0 y 4.0 con ácido acético 1 N ó hidróxido de amonio 6 N, según sea el caso, usando un pH-metro o un papel indicador de pH de rango estrecho (preferiblemente con límites de 2 a 5) como indicador externo. Diluir con agua hasta 40 mL y mezclar.

## Procedimiento

A cada uno de los tres tubos de comparación de color de 50 mL, que contienen respectivamente: la Preparación estándar, Preparación de prueba y Preparación control, adicionar 2 mL de Solución amortiguadora de acetato de pH 3.5. Luego adicionar 1.2 mL de Tioacetamida - glicerina base SR, diluir hasta 50 mL con agua y mezclar. Dejar en reposo durante 2 minutos y observar los tubos hacia abajo sobre una superficie blanca. El color de la solución de la Preparación de prueba no es más oscuro que el de la solución de la Preparación estándar y el color de la solución de la Preparación control es igual o más oscuro que el color de la solución de la Preparación estándar.

**Nota:** Si el color de la Preparación control es más claro que el de la Preparación estándar, utilizar el método II en lugar del método I para la sustancia en análisis.

**Criterio de aceptación:** El límite es 0.002%

## PUREZA-CROMATOGRAFICA (USP) (CoFa)

### Condiciones Cromatográficas

**Placa:** Placa cromatográfica en capa delgada recubierta con una capa de mezcla de sílica gel de 0.25 mm

**Fase móvil:** Tolueno: Tetrahidrofurano: Ácido acético Glacial (30:3:1)

## Se elimina la prueba

---

![img-7.jpeg](img-7.jpeg)

**ANEXO I**

**CONTROL DE CAMBIO SC-25-777**

**CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA**

**NAPROXENO SÓDICO**

**100000346**

Página 8 de 31

|  Volumen de Aplicación: 10 µL |   |
| --- | --- |
|  Desarrollo: ¼ partes de la longitud de la placa |   |
|  Solución Test |   |
|  Disolver 100.0 mg de muestra en 5 mL de Metanol. |   |
|  Solución Estándar |   |
|  Disolver 100.0 mg del Estándar de Naproxen Sódico en 5 mL de Metanol. |   |
|  Soluciones de Comparación |   |
|  Solución 1 (20 µg/mL): Tomar una alícuota de 10 µL de la Solución Estándar y transferir a una volumétrica de 10 mL, aforar a volumen con Metanol y mezclar. (0.1% de la Solución Estándar) |   |
|  Solución 2 (60 µg/mL): Tomar una alícuota de 30 µL de la Solución Estándar y transferir a una volumétrica de 10 mL, aforar a volumen con Metanol y mezclar. (0.3% de la Solución Estándar) |   |
|  Solución 3 (100 µg/mL): Tomar una alícuota de 50 µL de la Solución Estándar y transferir a una volumétrica de 10 mL, aforar a volumen con Metanol y mezclar. (0.5% de la Solución Estándar) |   |
|  Procedimiento |   |
|  Aplicar separadamente 10 µL de la Solución muestra, 10 µL de Solución estándar, 10 µL de Solución 1 de comparación, 10 µL de Solución 2 de comparación, y 10 µL de Solución 3 de comparación en la placa cromatográfica recubierta con una capa de 0.25 mm de gel de sílice y dejar secar las aplicaciones al aire. Colocar la placa en una cámara cromatográfica y desarrollar el cromatograma con |   |
|  fase móvil constituida por una mezcla de tolueno, tetrahidrofurano y ácido acetico-glacial (30:3:1) hasta que el frente de la fase móvil haya recorrido aproximadamente ¼ partes de la longitud de la placa. Retirar la placa de la cámara |   |

---

![img-8.jpeg](img-8.jpeg)

Página 9 de 31

de desarrollo, marcar el frente de la fase móvil y dejar secar al aire libre. Examinar la placa bajo luz UV con longitud de onda corta y localizar las manchas.

Criterio de aceptación: El valor Rf de la mancha principal del cromatograma de la solución en análisis corresponde con el de la solución estándar, la intensidad de cualquier mancha secundaria individual no excede la de la solución de comparación de 100 µg/mL (0.5%) y la suma de las intensidades de las manchas secundarias, comparadas de forma similar, no excede de 2.0%.

## VALORACIÓN (USP)

### p-naftolbenceína SR

Disolver 250.0 mg de p-naftolbenceína en 100 ml de ácido acético glacial.

### Procedimiento

Transferir 200.0 mg de muestra a un beaker y disolver con 50 mL de ácido acético Glacial, adicionar 2 gotas de p-naftolbenceína SR previamente neutralizado con ácido perclórico 0.1 N si es necesario y titular con ácido perclórico 0.1 N SV (o titular potenciométricamente hasta el punto de inflexión). Cada mL de ácido perclórico 0.1 N SV. Equívale a 25.22 mg de C₁₄H₁₁NaO₂ (Naproxen Sódico). Realizar un blanco para hacer las correcciones necesarias.

Calcular el porcentaje de Naproxen Sódico C₁₄H₁₁NaO₂ mediante la siguiente fórmula:

$$
\text{\% Naproxen Sódico} = \frac{(\text{Vm} - \text{Vb}) \times \text{F} \times 25.22 \times 100}{\text{Wm}}
$$

Donde:

Vm: Volumen de Acido perclórico 0.1N consumido en la titulación de la muestra.

Vb: Volumen de Acido perclórico 0.1N consumido en la titulación del blanco.

## VALORACIÓN (USP)

### Condiciones Cromatográficas

|  Condición | Calor  |
| --- | --- |
|  Columna: | XSelect HSS T3 Empaquetadura L1 (4.6 x 150) mm; 3.5 µm número de partes: 186004786 o equivalente  |
|  Detector UV: | 272 nm  |
|  Temperatura de la columna: | 35°C  |
|  Velocidad de flujo: | 1 mL/min  |
|  Volumen de inyección: | 20 µL  |
|  Fase móvil: | Solución A: Ácido acético glacial en Agua 0.1% Solución B: Ácido acético glacial en metanol 0.01%  |
|  Gradiente: | Ver Tabla 1. El tiempo de equilibrio recomendado después del gradiente es de 5 a 10 minutos.  |
|  Tiempo de retención: | Aproximadamente 16 minutos  |

---

![img-9.jpeg](img-9.jpeg)

Página 10 de 31

#### 100000346

#### 100000346 Wm: Peso de muestra (mg).

F: Factor de corrección de la concentración del titulante.

Criterio de aceptación: 98.0 - 102.0% calculado con respecto a la sustancia seca

#### Tabla 1 (Gradiente)

|  Tiempo (minutos) | Solución A (%) | Solución B (%)  |
| --- | --- | --- |
|  0 | 45 | 55  |
|  20 | 45 | 55  |
|  27 | 20 | 80  |
|  32 | 20 | 80  |
|  32.1 | 45 | 55  |
|  35 | 45 | 55  |

#### Soluciones

#### Solución A

Diluir 1 mL de ácido acético glacial en 1,000 mL de agua. Esta solución tiene una Concentración de 0.1%.

#### Solución B

Diluir 10 mL de ácido acético glacial en 100 mL de Metanol. Diluir un 1.0 mL de la solución anterior a una en 1,000 mL de Metanol. Esta solución tiene una Concentración de 0.01%.

#### Diluyente

Metanol: Agua (50:50)

#### Solución Estándar

Transferir aproximadamente 22 mg del Estándar de Naproxeno Sódico, pesados con exactitud, a una volumétrica de 20 mL, disolver hasta completa disolución, aforar a volumen en Diluyente y mezclar. Transferir una alícuota de 2.0 mL de la solución anterior a una volumétrica de 20 mL, diluir y aforar a volumen con Diluyente y mezclar. Pasar a vial por filtro jeringa PVDF (Polyvinylidene Fluoride) de 0.45 µm descartando los primeros 2.0 mL de filtrado. El procedimiento anterior se realiza por duplicado (dos pesadas independientes) y estas soluciones se rotulan como: Solución Estándar 1 y Solución Estándar 2.

Esta solución tiene una concentración de 0.11 mg/mL o 110 µg/mL de Naproxeno sódico.

---

# ANEXO 1

CONTROL DE CAMBIO SC-25-777 CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA NAPROXENO SODICO 100000346

Página 11 de 31

## Solución Muestra

Transferir aproximadamente 110 mg de la muestra, pesados con exactitud, a una volumétrica de $100-\mathrm{mL}$, disolver hasta completa disolución, aforar a volumen en Diluyente y mezclar. Transferir una alícuota de 10.0 mL de la solución anterior a una volumétrica de $100-\mathrm{mL}$, diluir y aforar a volumen con Diluyente y mezclar. Pasar a vial por filtro jeringa PVDF (Polyvinylidene Fluoride) de $0.45 \mu \mathrm{~m}$ descartando los primeros 2.0 mL de filtrado.

Esta solución tiene una concentración de $0.11 \mathrm{mg} / \mathrm{mL} 110 \mu \mathrm{~g} / \mathrm{mL}$ de Naproxeno sódico

## Procedimiento

Test de adecuabilidad del sistema Inyectar las soluciones según lo indicado en la Tabla 2 Tabla 2: Soluciones y número de inyecciones

|  Solución | Número de Inyeccion es | Parámetro a evaluar | Especificación  |
| --- | --- | --- | --- |
|  Diluyente | 1 | N.A | N.A  |
|  (SST Inicial)
Solución Estandar
(1 o 2) | 5 | Desviación Estándar
Relativa de las Áreas
(\%RSD) | El \%RSD de las áreas del pico de
Naproxeno Sódico es no más de
$0.73 \%$  |
|   |  | Factor de asimetría | El factor de asimetría para el pico de Naproxeno Sódico es no más de 2.0  |
|  Solución Estandar
(1) | 1 | Factor de correlación | El factor de Correlación debe estar entre $98.0 \%$ y $102.0 \%$  |
|  Solución Estandar
(2) | 1 | Valoración | $98.0 \%-102.0 \%$  |
|  Solución Muestra
(2 réplica) | 1 | Desviación Estándar
Relativa de las Áreas
(\%RSD) | El \%RSD entre las cinco áreas iniciales y el área final obtenidas  |
|  (SST Final)
Solución Estandar
(1 o 2) | 1 |  |   |

---

![img-10.jpeg](img-10.jpeg)

Página 12 de 31

|  (utilizado en el System inicial) |  | para el pico de Naproxeno Sódico es no más de 0.73%.  |
| --- | --- | --- |
|   | Factor de asimetría | El factor de asimetría para el pico de Naproxeno Sódico es no más de 2.0  |

### Cálculos

- Calcular el factor de correlación:

$$ \text{Factor de correlación} = 100 \times \frac{(A_2 \times W_1)}{(A_1 \times W_2)} $$

Dónde:

A1: Área del pico de Naproxeno Sódico en el cromatograma de la Solución Estándar 1.

A2: Área del pico de Naproxeno Sódico en el cromatograma de la Solución Estándar 2.

W1: peso de estandar de Naproxeno Sódico usado en la Solución Estándar 1. (mg)

W2: peso de estandar de Naproxeno Sódico usado en la Solución Estándar 2. (mg)

- Calcular el porcentaje de Naproxeno Sódico en base As is, en la porción de muestra tomada, mediante la siguiente fórmula:

$$ \% \text{Naproxeno Sódico base As is} = \left[ \frac{tru}{rs} \right] \times \left[ \frac{Ws \times 1}{20 \times 100 \times 20} \right] \times \left[ \frac{100 \times 100}{Wm \times 10} \right] \times 100 $$

Dónde:

ru: Área del pico de Naproxeno Sódico en el Cromatograma de la Solución Muestra

rs: Área del pico de Naproxeno Sódico en el Cromatograma de la Solución Estándar

---

![img-11.jpeg](img-11.jpeg)

**ANEXO 1**

**CONTROL DE CAMBIO SC-25-777**

**CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA**

**NAPROXENO SÓDICO**

**100000346**

Página 13 de 31

|  |   |
| --- | --- |
|   | Ws: Peso del Estándar de Naproxeno Sódico (mg)  |
|   | [ ]: Porcentaje de pureza del Estándar de Naproxeno Sódico (Base As is).  |
|   | Wm: Peso de la muestra (mg).  |
|   | - Calcular el % de la sustancia seca utilizar la siguiente formula:  |
|   | % Sustancia seca= $\frac{(\% \text{ Naproxeno sodico base As is x 100})}{(\text{100 - perdida por secado})}$  |
|   | Criterio de aceptación: 98.0% -102.0% con respecto a la sustancia seca  |
|  |   |
|   | **IMPUREZAS ORGÁNICAS (USP)**  |
|   | **Condiciones Cromatográficas**  |
|   | Columna:  |
|   | Detector DAD:  |
|   | Temperatura de la columna:  |
|   | Velocidad de flujo:  |
|   | Volumen de inyección:  |
|   | Soluciones  |
|   | Fase móvil  |
|   | Proceder como se indica en la valoración  |
|   | **Gradiente**  |
|   | Proceder como se indica en la valoración  |

---

![img-12.jpeg](img-12.jpeg)

Página 14 de 31

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777
### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA
### NAPROXENO SÓDICO
### 100000346

**Solución A y Solución B**
Proceder como se indica en la valoración

**Diluyente**
Metanol: Agua (50:50)

**Solución Stock Estándar de Naproxeno Sódico**
Transferir aproximadamente 22 mg del Estándar de Naproxeno Sódico, pesados con exactitud, a una volumétrica de 20-mL, disolver hasta completa disolución, aforar a volumen en Diluyente y mezclar.

Esta solución tiene una concentración de 1.1 mg/mL o 1.100 µg/mL de Naproxeno sódico.

**Solución Stock Estándar de Compuesto Relacionado K de Naproxeno**
Transferir aproximadamente 25 mg del Estándar de Compuesto Relacionado K de Naproxeno, pesados con exactitud, a una volumétrica de 50-mL, disolver hasta completa disolución, aforar a volumen en Diluyente y mezclar. Transferir una alícuota de 1.0 mL de la solución anterior a una volumétrica de 100-mL, diluir y aforar a volumen con Diluyente y mezclar.

Esta solución tiene una concentración de 0.005 mg/mL o 5.0 µg/mL de Compuesto Relacionado K de Naproxeno.

**Solución Stock Estándar de Compuesto Relacionado A de Naproxeno**
Transferir aproximadamente 20 mg del Estándar de Compuesto Relacionado A de Naproxeno, pesados con exactitud, a una volumétrica de 100-mL, disolver hasta completa disolución, aforar a volumen en Diluyente y mezclar. Transferir una alícuota de 5.0 mL de la solución anterior a una volumétrica de 20-mL, diluir y aforar a volumen con Diluyente y mezclar. De la solución anterior transferir una alícuota de 1.0 mL a una volumétrica de 10-mL, diluir y aforar a volumen con Diluyente y mezclar.

Esta solución tiene una concentración de 0.005 mg/mL o 5.0 µg/mL de Compuesto Relacionado A de Naproxeno.

---

![img-13.jpeg](img-13.jpeg)

Página 15 de 31

# ANEXO I

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO 100000346

##### Solución Stock Estándar de Compuesto Relacionado L de Naproxeno

Transferir aproximadamente 15 mg del Estándar de Compuesto Relacionado L de Naproxeno, pesados con exactitud, a una volumétrica de 25 mL, disolver hasta completa disolución, aforar a volumen en Diluyente y mezclar. Transferir una alícuota de 2.0 mL de la solución anterior a una volumétrica de 10 mL, diluir y aforar a volumen con Diluyente y mezclar. De la solución anterior transferir una alícuota de 4.0 mL a una volumétrica de 100 mL, diluir y aforar a volumen con Diluyente y mezclar.

Esta solución tiene una concentración de 0.0048 mg/mL o 4.8 µg/mL de Compuesto Relacionado L de Naproxeno.

##### Solución Stock Estándar de Compuesto Relacionado E de Naproxeno

Transferir aproximadamente 25 mg del Estándar de Compuesto Relacionado E de Naproxeno, pesados con exactitud, a una volumétrica de 50 mL, disolver hasta completa disolución, aforar a volumen en Diluyente y mezclar. Transferir una alícuota de 1.0 mL de la solución anterior a una volumétrica de 100 mL, diluir y aforar a volumen con Diluyente y mezclar.

Esta solución tiene una concentración de 0.005 mg/mL o 5.0 µg/mL de Compuesto Relacionado E de Naproxeno.

##### Solución de Idoneidad del Sistema

Transferir una alícuota de 2.0 mL de la Solución Stock Estándar de Naproxeno sódico, 2.0 mL de la Solución Stock Estándar de Compuesto Relacionado E de Naproxeno, 2.0 mL de la Solución Stock Estándar de Compuesto Relacionado A de Naproxeno y 2.0 mL de la Solución Stock Estándar de Compuesto Relacionado L de Naproxeno a una volumétrica de 20 mL, diluir y aforar a volumen con Diluyente y mezclar.

Esta solución tiene una concentración de Estándar de Naproxeno sódico de 0.11 mg/mL o 110 µg/mL, una concentración de Compuesto Relacionado E de Naproxeno de 0.0005 mg/mL o 0.5 mg/mL, una concentración de Compuesto Relacionado A de Naproxeno de 0.0005 mg/mL o 0.5 mg/mL, una concentración de Compuesto Relacionado L de Naproxeno de 0.00048 mg/mL o 0.48 mg/mL.

---

![img-14.jpeg](img-14.jpeg)

Página 16 de 31

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO

##### 100000346

###### Solución Estándar Intermedio de Naproxeno Sódico

Transferir una alícuota de 1.0 mL de la Solución Stock Estándar de Naproxeno Sódico a una volumétrica de 100 mL, diluir y aforar a volumen en diluyente y mezclar.

Esta solución tiene una concentración de 0.011 mg/mL o 11.0 µg/mL de Naproxeno Sódico.

###### Solución Estándar Mixto

Transferir una alícuota de 1.0 mL de la Solución Estándar Intermedio de Naproxeno Sódico a 2.0 mL de la Solución Stock Estándar de Compuesto Relacionado E de Naproxeno a 2.0 mL de la Solución Stock Estándar de Compuesto Relacionado A de Naproxeno a 2.0 mL de la Solución Stock Estándar de Compuesto Relacionado L de Naproxeno a una volumétrica de 100 mL, diluir y aforar a volumen con Diluyente y mezclar.

Esta solución tiene una concentración de Estándar de Naproxeno Sódico de 0.00011 mg/mL o 0.1 µg/mL, una concentración de Compuesto Relacionado E de Naproxeno de 0.0001 mg/mL o 0.1 µg/mL, una concentración de Compuesto Relacionado A de Naproxeno de 0.0001 mg/mL o 0.1 µg/mL, una concentración de Compuesto Relacionado L de Naproxeno de 0.000096 mg/mL o 0.096 de µg/mL.

###### Solución Estándar Intermedio de Sensibilidad de Naproxeno Sódico

Transferir una alícuota de 5.0 mL Solución Stock Estándar de Naproxeno Sódico a una volumétrica de 100 mL, diluir y aforar a volumen con Diluyente y mezclar. Transferir una alícuota de 1.0 mL de la solución anterior a una volumétrica de 10 mL, diluir y aforar a volumen con Diluyente y mezclar.

Esta solución tiene una concentración de 0.0055 mg/mL o 5.5 µg/mL de Naproxeno Sódico.

###### Solución de Sensibilidad

Transferir una alícuota de 1.0 mL de la Solución Estándar Intermedio de Sensibilidad de Naproxeno a 1.0 mL de la Solución Stock Estándar de Compuesto Relacionado A de Naproxeno a 1.0 mL de la Solución Stock Estándar de Compuesto Relacionado L de Naproxeno a una volumétrica de 100 mL, diluir y aforar a volumen con Diluyente y mezclar.

---

![img-15.jpeg](img-15.jpeg)

Página 17 de 31

![img-16.jpeg](img-16.jpeg)

**ANEXO 1**

**CONTROL DE CAMBIO SC-25-777**

**CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA**

**NAPROXENO SÓDICO**

**100000346**

Esta solución tiene una concentración de Naproxeno sódico de 0.000055 mg/mL o 0.055 µg/mL, una concentración de Compuesto Relacionado A de Naproxeno de 0.000055 mg/mL o 0.055 µg/mL, una concentración de Compuesto Relacionado L de Naproxeno de 0.000048 mg/mL o 0.048 µg/mL, una concentración de Compuesto Relacionado E de Naproxeno de 0.000055 mg/mL o 0.055 µg/mL.

**Solución Muestra**

Transferir aproximadamente 110 mg de la muestra, pesados con exactitud, a una volumétrica de 100-mL, disolver hasta completa disolución, aforar a volumen en Diluyente y mezclar. Transferir una alícuota de 10.0 mL de la solución anterior a una volumétrica de 100-mL, diluir y aforar a volumen con Diluyente y mezclar. Pasar a vial por filtro jeringa PVDF (Polyvinylidene Fluoride) de 0.45 µm descartando los primeros 2.0 mL de filtrado.

Esta solución tiene una concentración de 0.11 mg/mL o 110 µg/mL de Naproxeno sódico procedimiento

**NOTA 1:** A manera de información en la Tabla 3 se proporcionan los tiempos de retención relativos.

**Tabla 3 Tiempos de retención relativos**

|  Nombre | Tiempo de retención relativo | Criterios de aceptación (%)  |
| --- | --- | --- |
|  Compuesto relacionado K del naproxeno, si está presente | 0.66 | 0.10  |
|  Compuesto relacionado A del naproxeno | 0.71 | 0.06  |
|  Compuesto relacionado L del naproxeno | 0.88 | 0.1  |
|  naproxeno | 1.0 | —  |
|  Compuesto relacionado E del naproxeno | 1.63 | 0.1  |
|  Cualquier impureza no especificada | — | 0.10  |
|  Impurezas totales | — | 0.5  |

**Test de adecuabilidad del sistema**

*Inyectar las soluciones según lo indicado en la Tabla 4*

---

![img-17.jpeg](img-17.jpeg)

Página 18 de 31

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA NAPROXENO SODICO 100000346

|  Tabla 4: Soluciones y número de inyecciones | Número de Inyección | Parámetro a evaluar | Especificación  |
| --- | --- | --- | --- |
|  Diluyente | 1 | N.A | N.A  |
|  Solución de sensibilidad | 1 | Relación señal-ruido | La relación/senal ruido es no menos de 10 para cada pico  |
|  Solución de Idoneidad del sistema | 1 | Resolución | La Resolución entre el Compuesto Relacionado K de Naproxeno y el Compuesto Relacionado A de Naproxeno es no menos de 1.5  |
|  (SST Inicial) Solución Estándar mixto | 6 | Desviación Estándar Relativa (%RSD) | El %RSD de las áreas para cada pico es no más de 5.0%  |
|  Diluyente | 1 | N.A | N.A  |
|  Solución Muestra (1 réplica) | 1 | Impurezas | Compuesto Relacionado K del Naproxeno, si está presente: No más de 0.10%  |
|  Diluyente | 1 | N.A | Compuesto Relacionado A del Naproxeno: No más de 0.06%  |
|  Solución Estándar mixto (utilizado en el System inicial) | 1 | Desviación Estándar Relativa (%RSD) | El %RSD entre las seis áreas iniciales y el área final obtenidas para cada pico es no más de 5.0%  |

---

![img-18.jpeg](img-18.jpeg)

Página 19 de 31

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO 100000346

#### Cálculos

- Calcular el porcentaje del Compuesto Relacionado E de Naproxeno en la porción de Naproxeno Sódico tomada:

$$
\text{\% Compuesto Relacionado E} = \left( \frac{\text{rB}}{\text{rs}} \right) \times \left( \frac{\text{Ws} \times 1}{\text{50} \times 100} \times 100 \times 100 \right) \times \left( \frac{100 \times 100}{\text{Wm} \times 10} \right) \times 100
$$

Dónde:

- **r1**: Área del Compuesto Relacionado E de Naproxeno, de la Solución Muestra
- **r5**: Área del Compuesto Relacionado E de Naproxeno, de la Solución Estándar
- **Ws**: Peso del Compuesto Relacionado E de Naproxeno (mg)
- **l**: Porcentaje de pureza del Compuesto Relacionado E de Naproxeno (Base As Is)
- **Wm**: Peso de la muestra (mg)
- Calcular el porcentaje del Compuesto Relacionado A de Naproxeno en la porción de Naproxeno Sódico tomada:

$$
\text{\% Compuesto Relacionado A} = \left( \frac{\text{rB}}{\text{rs}} \right) \times \left( \frac{\text{Ws} \times 1}{\text{100} \times 100} \times 20 \times 10 \times 100 \right) \times \left( \frac{100 \times 100}{\text{Wm} \times 10} \right) \times 100
$$

Dónde:

- **r1**: Área del Compuesto Relacionado A de Naproxeno de la Solución Muestra
- **r5**: Área del Compuesto Relacionado A de Naproxeno de la Solución Estándar
- **Ws**: Peso del Estándar de Compuesto Relacionado A de Naproxeno (mg)

---

![img-19.jpeg](img-19.jpeg)

Página 21 de 31

|   | Ws: Peso del Estándar de Naproxeno sódico (mg)  |
| --- | --- |
|   | [ ]: Porcentaje de pureza del Estándar de Naproxeno sódico en (Base As Is)  |
|   | Wm: Peso de la muestra (mg)  |
|   | F: Factor de respuesta relativo, 0.71 para el Compuesto Relacionado K de Naproxeno y 1.0 para la Impurezas No especificadas  |
|   | **NOTA:** El umbral de reporte de notificación es del 0.05%.  |
|   | Criterios de Aceptación:  |
|   | Compuesto Relacionado K de Naproxeno, si está presente: No más de 0.10%.  |
|   | Compuesto Relacionado A de Naproxeno: No más de 0.06%.  |
|   | Compuesto Relacionado L de Naproxeno: No más de 0.1%.  |
|   | Compuesto Relacionado E de Naproxeno: No más de 0.1%.  |
|   | Cualquier Impureza No Especificada: No más de 0.10%.  |
|   | Impurezas Totales: No más de 0.5%  |
|   | **PRUEBA DE PUREZA ENANTIOMERICA (USP)**  |
|   | Condiciones Cromatográficas  |
|   | **Columna:**  |
|   | **Detector DAD:**  |
|  **Se Incluye La Prueba descrita en la monografía USP** | **RegisPack, (S,S) Whelk-O L102 (4.6 x 250) mm; 5 µm**  |
|   | **Numero de partes 1-780101-300**  |
|   | **263 nm**  |
|   | **25°C**  |
|   | **2 mL/min**  |
|   | **Volumen de inyección:**  |
|   | **20 µL**  |

---

![img-20.jpeg](img-20.jpeg)

Página 22 de 31

|  Tiempo de corrida: | No menos de 1.5 veces el tiempo de retención del pico de Naproxeno  |
| --- | --- |
|  Fase móvil: | Hexano: Isopropanol: Acetonitrilo: Ácido acético glacial (84.5:10:5:0.5)  |

**Soluciones**

*NOTA: Proteger las soluciones que contienen naproxeno de la luz*

### *Solución Stock Estandar de Compuesto Relacionado G de Naproxeno*

Transferir aproximadamente 10 mg del estandar de Compuesto Relacionado G de Naproxeno, pesados con exactitud, a una volumétrica de 200-mL disolver hasta completa disolución, aforar a volumen con Fase Móvil y mezclar.

**Esta solución tiene una concentración de 0.05 mg/mL o 50 µg/mL de Compuesto Relacionado G de Naproxeno**

### *Solución Stock Estandar de Naproxeno Sódico*

Transferir aproximadamente 12.5 mg del Estandar de Naproxeno Sódico, pesados con exactitud, a una volumétrica de 50-mL disolver hasta completa disolución, aforar a volumen con Fase Móvil y mezclar.

**Esta solución tiene una concentración de 0.25 mg/mL o 250 µg/mL de Naproxeno Sódico**

### *Solución de aptitud del Sistema*

Transferir una alícuota de 5.0 mL de la *Solución Stock de Compuesto Relacionado G de Naproxeno* y 1.0 mL de *Solución Stock de Naproxeno Sódico* a una volumétrica de 10-mL, disolver y aforar a volumen con fase móvil y mezclar. Pasar a vial por filtro jeringa PVDF (Polyvinylidene Fluoride) de 0.45 µm descartando los primeros 2.0 mL de filtrado.

**Esta solución tiene una concentración de 0.025 mg/mL o 25 µg/mL de Compuesto Relacionado G de Naproxeno y 0.025 mg/mL o 25 µg/mL de Naproxeno Sódico.**

---

![img-21.jpeg](img-21.jpeg)

Página 23 de 31

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO

##### 100000346

###### Solución Estandar de Compuesto Relacionado G

Transferir una alícuota de 5.0 mL de la Solución Stock de Compuesto Relacionado G de Naproxeno a una volumétrica de 10-mL diluir y aforar a volumen en Fase Móvil y mezclar, de la solución anterior tomar una alícuota de 1.0 mL y transferirlo a una volumétrica de 20-mL, diluir y aforar a volumen en Fase Móvil y mezclar.

Esta solución tiene una concentración de 0.00125 mg/mL o 1.25 µg/mL de Compuesto Relacionado G de Naproxeno.

###### Solución Stock Muestra

Equivalente a 500 µg/mL de naproxeno en tetrahidrofurano preparado de la siguiente manera: En un recipiente adecuado Disolver aproximadamente 25 mg de la muestra en 15 mL de agua y agregar 1 mL de ácido clorhídrico. Transferir la solución resultante a un embudo de separación y añadir 10 mL de Acetato de etilo y agitar bien para extraer el naproxeno libre. Separar la capa de acetato de etilo y repetir la extracción con 10 mL adicionales de acetato de etilo. Combinar las dos extracciones de acetato de etilo y evaporar hasta sequedad al vacío. Disolver el residuo en 50 mL de tetrahidrofurano.

Esta solución tiene una concentración de 0.5 mg/mL o 500 µg/mL de Naproxeno Sódico.

###### Solución Muestra

Transferir una alícuota de 1.0 mL de Solución Stock Muestra a una volumétrica de 10-mL, diluir y aforar a volumen con Fase móvil y mezclar. Pasar a vial por filtro jeringa PVDF (Polyvinylidene Fluoride) de 0.45 µm descartando los primeros 2.0 mL de filtrado.

Esta solución tiene una concentración de 0.05 mg/mL o 50 µg/mL de Naproxeno Sódico.

###### Procedimiento

Test de adecuabilidad del sistema

*Inyectar las soluciones según lo indicado en la Tabla 5*

**NOTA:** A manera de información los tiempos de retención relativos para compuesto relacionado G de naproxeno y naproxeno son aproximadamente 0.7 y 1.0, respectivamente.

---

![img-22.jpeg](img-22.jpeg)

Página 24 de 31

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO 100000346

|  Tabla 5: Soluciones y número de inyecciones | Número de Inyecciones | Parámetro a evaluar | Especificación  |
| --- | --- | --- | --- |
|  Fase móvil | 1 | N.A | N.A  |
|  Solución aptitud del sistema | 1 | Resolución R | La Resolución entre el pico del Compuesto Relacionado G de Naproxeno y el Naproxeno es no menos de 3.0  |
|  (SST inicial) Solución Estándar de Compuesto Relacionado G | 5 | Desviación Estándar Relativa (%RSD) | El %RSD de las áreas del pico del Compuesto Relacionado G de Naproxeno es no más de 2.0%.  |
|  Fase móvil | 1 | N.A | N.A  |
|  Solución Muestra (2 réplica) | 1 | Pureza Enantiomérica | No más de 2.5%  |
|  (SST final) Solución Estándar de Compuesto Relacionado G (utilizado en el System inicial) | 1 | Desviación Estándar Relativa (%RSD) | El %RSD entre las cinco áreas iniciales y el área final obtenidas para el pico del Compuesto Relacionado G de Naproxeno es no más de 2.0%.  |

#### Cálculos

Calcular el porcentaje del Compuesto Relacionado G de Naproxeno en la porción de Naproxeno Sódico tomada:

$$ \% \text{ de Compuesto Relacionado G} = \left[ \frac{70}{15} \right] \times \left[ \frac{50 \times 10}{200 \times 100 \times 10 \times 20} \right] \times \left[ \frac{50 \times 10}{100 \times 100 \times 100} \right] \times 100 $$

---

![img-23.jpeg](img-23.jpeg)

Página 25 de 31

|   | Dónde: |   |
| --- | --- | --- |
|   | ru: Área del pico del Compuesto Relacionado G de Naproxeno de la Solución Muestra |   |
|   | rₑ: Área del pico del Compuesto Relacionado G de Naproxeno de la Solución Estándar |   |
|   | Ws: Peso del Estándar de Compuesto Relacionado G de Naproxeno (mg) |   |
|   | [ ]: Porcentaje de pureza del Estándar Compuesto Relacionado G de Naproxeno en (Base As Is) |   |
|   | Wm: Peso de la muestra (mg) |   |
|   | Criterios de Aceptacion: No más de 2.5% |   |
|  SOLVENTES RESIDUALES (USP) (CoFa) | SOLVENTES RESIDUALES (USP) |   |
|  Condiciones Cromatográficas | Condiciones Instrumentales |   |
|  Columna: | Columna capilar de sílice fundida de 0.32 mm × 30 m, recubierta con una película de 1.8 µm de G43 (6% cianopropilfenil - 94% dimetilpolisiloxano). | Columna capilar de sílice fundida de 0.32 mm × 30 m, recubierta con una película de 1.8 µm de G43 (6% cianopropilfenil - 94% dimetilpolisiloxano) J&W DB-624 o equivalente  |
|  Gas de Arrastre: | Nitrógeno o Helio |   |
|  Velocidad Lineal: | 35 cm/ segundo | FID, 250 °C  |
|  Volumen de Inyección: | 1 µL | Temperatura del Puerto de Inyección:  |
|  Detector: | Ionizador a la Llama (FID) | Gas de Arrastre:  |
|  Relación de partición: | 1:5 | Flujo:  |
|  Gas de Arrastre: |  | Relación de partición:  |
|  Velocidad Lineal: |  | Temperatura de la Columna:  |
|  Gas de Arrastre: |  | 40°C durante 20 minutos, luego aumentar 10°C por minuto h₂  |
|  Volumen de Inyección: |  | 240°C y mantener hasta 240°C por 20 minutos.  |
|  Detector: |  |   |

---

![img-24.jpeg](img-24.jpeg)

**ANEXO 1**

**CONTROL DE CAMBIO SC-25-777**

**CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA**

**NAPROXENO SÓDICO**

**100000346**

Página 26 de 31

|  Temperatura del Puerto de Inyección: | 140°C  |
| --- | --- |
|  Temperatura del Detector: | 250°C  |
|  Temperatura de la Columna: | Temperatura inicial: 40°C durante 20 minutos  |
|  Temperatura final: | 240°C durante 20 minutos  |
|  Solución Madre del Estándar de Clase 1 |   |
|  Nota: Al transferir las soluciones colocar la punta de la pipeta justo por debajo de la superficie del líquido y mezclar. |   |
|  Transferir 1mL de Mezcla de Solventes Residuales Clase 1 USP, medidos con exactitud a una volumétrica de 100 mL, a la que previamente se han agregado aproximadamente 9 mL de Dimetil-sulfoxide, diluir con agua a volumen y mezclar. Transferir una alícuota de 1 mL de la solución anterior a una volumétrica de 100 mL a la que previamente se le han adicionado 50 mL de agua, aforar a volumen con agua y mezclar. |   |
|  Solución Estándar de Clase 1 |   |
|  Nota: Al transferir las soluciones colocar la punta de la pipeta justo por debajo de la superficie del líquido y mezclar. |   |
|  Transferir 1mL de la Solución Madre del Estándar Clase 1 a un vial para muestreo de fase gaseosa apropiado que contenga 5 mL de agua tapar y mezclar. |   |
|  Solución Madre del Estándar Mezcla A de Clase 2 |   |

|  Tiempo de corrida: | 60 minutos  |
| --- | --- |
|  Flujo de Hidrogeno: | 30 mL/min  |
|  Velocidad Lineal: | 35 cm/segundo  |
|  Volumen de Inyección: | 1 µL  |
|  Flujo de Aire: | 400 mL/min  |
|  Condiciones del Sistema Headspace |   |
|  Temperatura del vial: | 80°C  |
|  Tiempo de Equilibrio: | 45 minutos  |
|  Temperatura del Loop: | 85°C  |
|  Temperatura Lineal: | 105°C  |
|  Gas de arrastre: | Helio  |
|  Tiempo de Presurización: | 60 segundos  |
|  Tiempo de Equilibrio del Loop: | 0.05 minutos  |
|  Tiempo de inyección: | 2 minutos  |
|  Tiempo de llenado del Loop: | 2 minutos  |
|  Volumen de Inyección: | 1.0 mL  |
|  Tiempo ciclo cromatográfico: | 90 minutos  |
|  Agitación: | baja, 1 minuto  |
|  Presión del Vial: | 10 psi  |
|  Tamaño del vial: | 10 mL  |

---

![img-25.jpeg](img-25.jpeg)

Página 27 de 31

**ANEXO I**

**CONTROL DE CAMBIO SC-25-777**

**CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA**

**NAPROXENO SODICO**

**100000346**

**Nota:** Al transferir las soluciones colocar la punta de la pipeta justo por debajo de la superficie del líquido y mezclar.

Transferir 1 mL de Mezcla A de Solventes Residuales Clase 2 USP, medidos con exactitud a una volumétrica de 100 mL, diluir con agua a volumen y mezclar.

**Solución Estándar Mezcla A de Clase 2**

Transferir 1 mL de la Solución Madre del Estándar Mezcla A Clase 2, a un vial para muestreo de fase gaseosa apropiado, agregar 5 mL de agua tapar y mezclar.

**Solución Madre del Estándar Mezcla B de Clase 2**

Transferir 1 mL de Mezcla B de Solventes Residuales Clase 2 USP, medidos con exactitud a una volumétrica de 100 mL, diluir con agua a volumen y mezclar.

**Solución Estándar Mezcla B de Clase 2**

Transferir 5 mL de la Solución Madre del Estándar Mezcla B Clase 2, a un vial para muestreo de fase gaseosa apropiado, agregar 1 mL de agua tapar y mezclar.

**Solución Madre de la Muestra**

Transferir aproximadamente 250.0 mg del producto, pesado con exactitud, a una volumétrica de 25 mL, disolver, aforar a volumen con agua y mezclar.

**Solución Muestra**

Transferir 5 mL de la Solución Madre de la Muestra, a un vial para muestreo de fase gaseosa apropiado, agregar 1 mL de agua tapar y mezclar.

**Solución de Aptitud del Sistema de Clase 1**

Transferir 1 mL de Solución de la solución Madre del Estándar de Clase 1 a un vial para muestreo de fase gaseosa adecuado, agregar 5 mL de la Solución Madre de la Muestra, tapar y mezclar.

**Procedimiento**

**Solución Stock Estándar de Clase 1**

**NOTA 4:** Al transferir las soluciones, colocar la punta de la pipeta justo por debajo de la superficie del líquido y mezclar.

Transferir 1.0 mL de estándar mezcla de Solventes Residuales - Clase 1 USP a una volumétrica de 100 mL previamente llena con aproximadamente 9.0 mL de Dimetilsulfóxido (DMSO), diluir a volumen con agua a volumen y mezclar.

Transferir 1.0 mL de esta solución a una volumétrica de 100 mL, previamente llenado con aproximadamente 50 mL de agua, diluir a volumen con agua a volumen y mezclar.

Transferir 10 mL de esta solución a un matraz volumétrico de 100 mL que contenga aproximadamente 50 mL de agua, diluir con agua a volumen y mezclar.

**Solución Estándar de Clase 1**

Transferir 1.0 mL de la Solución Stock Estándar de Clase 1 a un vial de headspace de 10 mL, que contiene 5.0 mL de agua, aplique el tapón, tape y mezcle.

**Solución Stock Estándar Clase 2 Mezcla A**

Transferir 1.0 mL de Estándar mezcla A de Solventes Residuales Clase 2 USP a una volumétrica de 100 mL, diluir a volumen con agua y mezclar.

**Solución Estándar Mezcla A de Clase 2**

Transferir 1.0 mL de Solución Stock Estándar Clase 2 Mezcla A a un vial de headspace de 10 mL, que contiene 5.0 mL de agua, aplique el tapón, tape y mezcle.

**Solución Stock Estándar (Metanol)**

Transferir 1.0 mL de Residual Solvent Class 2-Metanol USP a una volumétrica de 100 mL, previamente llena con 15 mL de agua. Aforar a volumen con agua y Mezclar.

**Solución Stock Estándar (Tolueno)**

Transferir 1.0 mL Residual Solvent Class 2-Tolueno USP a una volumétrica de 100 mL, previamente llena con 15 mL de agua. Aforar a volumen con agua y Mezclar.

---

![img-26.jpeg](img-26.jpeg)

Página 28 de 31

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO 100000346

A.— Inyectar la Solución Estándar Clase 1 y verificar que:

- La señal ruido del 1, 1, 1 Tricloroetano no es menor de 5

B.— Inyectar la Solución Aptitud del Sistema Clase 1

- La señal ruido de cada pico no es menor de 3

C.— Inyectar la Solución Estándar Mezcla A Clase 2

- La resolución, R, entre acetonitrilo y cloruro de metileno no es menor de 1.0

**Nota:** Se recomienda incrementar la temperatura de la línea de transferencia entre corridas para eliminar cualquier condensación potencial de los disolventes.

D.— Inyectar por separado en el cromatograma, volúmenes iguales de la fase gaseosa (aproximadamente 1 mL), siguiendo alguno de los sets de parámetros operativos para inyectar de fase gaseosa descritos en la tabla 1.

- Solución Blanco (Agua)
- Solución Estándar Clase 1
- Solución Estándar Mezcla A de Clase 2
- Solución Estándar Mezcla B de Clase 2
- Solución de Lavado (Agua)
- Solución Muestra

**Tabla 1**

|  Parámetros Operativos para el Inyector de fase Gaseosa | 1 | 2 | 3  |
| --- | --- | --- | --- |
|  Temperatura de Equilibrio (°) | 80 | 105 | 80  |

#### Solución Estándar

Por separado, transferir 1.0 mL de cada una de las Soluciones Stock Estándar a un vial de headspace de 10 mL, que contenga 5.0 mL de agua, aplique el tapón, tape y mezcle.

#### Solución Stock Muestra

Transferir aproximadamente 500 mg de muestra pesados con exactitud, a una volumétrica de 50 mL, disolver y diluir con agua a volumen y mezclar.

#### Solución Muestra

Transferir 5.0 mL de la Solución Stock Muestra a un vial de headspace de 10 mL, que contiene 1.0 mL de agua, aplique el tapón, tape y mezcle.

#### Solución de Adecuabilidad del Sistema Clase 1

Mezclar 1.0 mL de Solución Stock Estándar de Clase 1 a un vial de headspace de 10 mL que contenga 5.0 mL Solución Stock Muestra, aplique el tapón, tape y mezcle.

#### Solución Muestra Enriquecida

Transferir 5 mL de la Solución Stock Muestra al vial de headspace de 10 mL, agregar 1 mL de la Solución Stock Estándar. Tapar y mezclar.

**NOTA 5:** Preparar una solución stock muestra enriquecida por separado para cada solvente a cuantificar.

#### Blanco

Transferir 6 mL de agua a un vial Headspace de 10 mL, agrafar.

#### Procedimiento

Test de adecuabilidad del sistema

Inyectar las soluciones según lo indicado en la Tabla 6.

---

![img-27.jpeg](img-27.jpeg)

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SODICO 100000346

Página 29 de 31

|  Tiempo de Equilibrio (minutos) | 60 | 45 | 45  |
| --- | --- | --- | --- |
|  Temperatura de línea de Transferencia (°) (si corresponde) | 85 | 110 | 105  |
|  Temperatura de la jeringa (°) (si corresponde) | 80-90 | 105-115 | 80-90  |
|  Gas transportador: nitrógeno o helio a una presión adecuada |  |  |   |
|  Tiempo de presurización (s) (si corresponde) | ≥60 | ≥60 | ≥60  |
|  Volumen de Inyección (mL) | 1 | 1 | 1  |

Si la respuesta de cualquier pico diferente del pico de 1,1,1-tricloroetano en la Solución Muestra es mayor o igual a la del pico correspondiente en la Solución Estándar Clase o en cualquiera de las dos Soluciones Estándar Mezcla de Clase 2.

### Criterios de Aceptación: Cumple requerimientos

|  Tabla 6 Soluciones y número de inyecciones | Número de Inyección | Parámetro a evaluar | Especificación  |
| --- | --- | --- | --- |
|  Blanco de diluyente | 1 | N.A | N.A  |
|  Solución estándar clase 1 | 1 | Señal/ruido | La relación señal / ruido para el pico de 1,1,1-tricloroetano es no menor de 5  |
|  Solución de adecuabilidad del sistema clase 1 | 1 | Señal/ruido | La relación señal / ruido para cada pico no es menor de 3  |
|  Solución estándar clase 2 mezcla A | 1 | Resolución | La resolución entre el acetonitrilo y el cloruro de metileno no es menor de 1.0  |
|  Solución estándar | 1 | Identificación de picos | Identificar los picos principales y medir las áreas para los picos principales  |
|  Solución estándar | 1 | Señal/ruido | La relación señal/ruido es de no menos de 10 para cualquier solvente que no sea de Clase 1, en el cromatograma de las soluciones estándar  |
|  Solución muestra (1 réplica) | 1 | Cuantificación Solvente Residual | Tolueno: No más de 890 ppm Metanol: No más de 3,000 ppm  |
|  Solución muestra enriquecida (1 réplica) | 1 | Cuantificación Solvente Residual | N.A  |
|  Solución estándar clase 1 | 1 | Resolución | La resolución entre un pico de interés y cualquier pico adyacente es de no menos de 1.0  |
|  Solución estándar clase 1 | 1 | Señal ruido | La relación señal / ruido para el pico de 1,1,1-tricloroetano es no menor de 5  |

---

![img-28.jpeg](img-28.jpeg)

Página 30 de 31

# ANEXO 1

## CONTROL DE CAMBIO SC-25-777

### CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA

#### NAPROXENO SÓDICO 100000346

#### Cálculos

- Calcular la relación señal / ruido para la *Solución estándar clase 1*, la *Solución de adecuabilidad del sistema clase 1* y la *Solución estándar clase 2 mezcla A* mediante la siguiente fórmula:

$$(2H) / h$$

- \( H \) = altura del pico de 1,1,1-tricloroetano
- \( h \) = ruido

- Cálculos para la *Solución Stock Estándar (Metanol)* y *Solución Stock Estándar (Tolueno)*

$$C \, (\mu g/mL) = \frac{1 \times L \times 1,000}{100}$$

Donde,

- \( L \): Concentración de Metanol o Tolueno en Residual Solvent Class 2 de Metanol o Tolueno (mg/mL).

Calcular la cantidad, en ppm, de Metanol y Tolueno mediante la siguiente fórmula:

$$(\frac{50}{5}) \times (\frac{C}{W}) \times [\frac{r_o}{(r_{ST} - r_o)}]$$

Donde:

- \( C \) = Concentración en µg por mL, del disolvente apropiado en la *Solución Stock Estándar*.

- \( W \) = Peso en gramos, de la muestra tomada para preparar la *Solución Stock Muestra*.

- \( r_{ST} \) = Área de respuesta para cada pico obtenido de la *Solución Muestra*.

- \( r_{ST} \) = Área de respuesta para cada pico obtenido a partir de la *Solución Muestra Enriquecida*.

---

![img-29.jpeg](img-29.jpeg)

**ANEXO I**

**CONTROL DE CAMBIO SC-25-777**

**CUADRO COMPARATIVO DE PRUEBAS PARA LA MATERIA PRIMA**

**NAPROXENO SODICO**

**100000346**

Página 31 de 31

50/5= Volumen (mL) de Solución Stock Muestra, dividido entre los 5 mL utilizados en la solución muestra

Criterio de aceptación:
- Tolueno: No más de 890 ppm
- Metanol: No más de 3,000 ppm

Elaborado por: ______________________

Fecha: 2021-08-28

Aprobado por: ______________________

Fecha: 2021-08-28

## Ejemplo 4: Definir un modelo personalizado

In [ ]:
# Define tu propio modelo Pydantic
class MiModeloPersonalizado(BaseModel):
    """Modelo personalizado para extracción."""
    titulo: Optional[str] = Field(None, description="Título del documento")
    fecha: Optional[str] = Field(None, description="Fecha del documento")
    autor: Optional[str] = Field(None, description="Autor del documento")
    resumen: Optional[str] = Field(None, description="Resumen del contenido")
    secciones: Optional[List[str]] = Field(None, description="Lista de secciones principales")
    conclusiones: Optional[str] = Field(None, description="Conclusiones del documento")

# Ver campos
show_model_fields(custom_model=MiModeloPersonalizado)

In [ ]:
# ⚠️ MODIFICA ESTA RUTA con tu documento
DOCUMENT_PATH = r"C:\ruta\a\tu\documento.pdf"

# Usar modelo personalizado
results_custom = test_document_annotation(
    document_path=DOCUMENT_PATH,
    custom_model=MiModeloPersonalizado,
    max_pages_per_chunk=8,
    save_output=True
)

In [ ]:
show_results(results_custom)

## Ejemplo 5: Comparar diferentes configuraciones de chunking

In [ ]:
# ⚠️ MODIFICA ESTA RUTA con tu documento
DOCUMENT_PATH = r"C:\ruta\a\tu\documento.pdf"

configs = [
    {"max_pages_per_chunk": 4, "chunk_overlap_pages": 1},
    {"max_pages_per_chunk": 8, "chunk_overlap_pages": 2},
    {"max_pages_per_chunk": 12, "chunk_overlap_pages": 3},
]

comparison_results = []

for config in configs:
    print(f"\n🔄 Probando config: {config}")
    result = test_document_annotation(
        document_path=DOCUMENT_PATH,
        model_name="analytical_method",
        save_output=False,
        **config
    )
    comparison_results.append({
        "config": config,
        "chunks": result.get("chunks_processed"),
        "time": result.get("elapsed_time_seconds"),
        "success": result.get("success")
    })

print("\n📊 COMPARACIÓN DE CONFIGURACIONES:")
print("-" * 60)
for r in comparison_results:
    print(f"Config: {r['config']} | Chunks: {r['chunks']} | Tiempo: {r['time']}s | Éxito: {r['success']}")

## Ejemplo 6: Acceder a datos específicos del modelo

In [ ]:
# Después de ejecutar un test, puedes acceder a campos específicos
if 'results' in dir() and results.get("success"):
    data = results["consolidated_data"]
    
    # Ejemplo para ChangeControlModel
    if results.get("model_used") == "ChangeControlModel":
        print("📋 Código de solicitud:", data.get("codigo_solicitud"))
        print("📅 Fecha:", data.get("fecha_solicitud"))
        print("📝 Título:", data.get("titulo"))
        
        if data.get("descripcion_cambio"):
            print(f"\n🔄 Cambios encontrados: {len(data['descripcion_cambio'])}")
            for i, cambio in enumerate(data["descripcion_cambio"][:3], 1):
                print(f"   {i}. {cambio.get('prueba', 'N/A')}: {cambio.get('texto', 'N/A')[:100]}...")
    
    # Ejemplo para MetodoAnaliticoDA
    elif results.get("model_used") == "MetodoAnaliticoDA":
        print("📋 Nombre producto:", data.get("nombre_producto"))
        print("📄 Número método:", data.get("numero_metodo"))
        print("📌 Versión:", data.get("version_metodo"))
        print("🎯 Objetivo:", data.get("objetivo", "")[:200] + "..." if data.get("objetivo") else "N/A")
else:
    print("⚠️ Ejecuta primero un test para ver los datos")

---

## 📚 Referencia Rápida

### Modelos disponibles:
- `change_control`: Para documentos de Control de Cambios
- `analytical_method`: Para métodos analíticos (sin markdown)
- `analytical_method_complete`: Para métodos analíticos (con markdown completo)
- `side_by_side`: Para comparativos lado a lado
- `test_solutions`: Para pruebas/tests analíticos

### Parámetros importantes:
- `max_pages_per_chunk`: Máximo de páginas por chunk (default: 8)
- `chunk_overlap_pages`: Páginas de solapamiento entre chunks (default: 2)
- `max_workers`: Número de workers para procesamiento paralelo (default: 4)
- `include_image_base64`: Incluir imágenes en base64 (default: False)

### Tips:
1. Para documentos muy largos, aumenta `max_pages_per_chunk` para reducir el número de llamadas API
2. El `chunk_overlap_pages` ayuda a no perder información en los bordes de los chunks
3. Usa `save_output=True` para guardar los resultados y revisarlos después
4. Define modelos personalizados para extraer exactamente lo que necesitas